## Import libraries

In [74]:
from pytwitter import Api
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import math
from pytwitter.models import User;
from pytwitter.models import Tweet;


In [273]:
api = Api(bearer_token="AAAAAAAAAAAAAAAAAAAAAMhqlAEAAAAA4Pqzn354Z5nlkP5lKaW98vzlVlA%3D7GIA03xacVKdFYTFg7qmgvWTZThpa2FFd4SNPUqP7uPK7Xjue5")

public_tweets = api.search_tweets(query="carnaval is:verified lang:pt has:hashtags -is:retweet", expansions=["referenced_tweets.id.author_id","in_reply_to_user_id","attachments.media_keys","author_id","entities.mentions.username"], 
                                  user_fields=["created_at","entities","id","location","name","pinned_tweet_id","profile_image_url","protected","public_metrics","url","username","verified"],
                                  tweet_fields=["attachments","author_id","context_annotations","created_at","entities","geo","in_reply_to_user_id","lang","public_metrics","reply_settings","source"], max_results=20)

# Analysis

## User's influence and reputation Analysis

In [68]:
def get_sentiment_score(text):
    "float: The sentiment score between -1.0 (negative) and 1.0 (positive)"
    blob = TextBlob(text)
    return blob.sentiment.polarity

### Influence Calculus

In [84]:
def calculate_influence(user: User):
    follower_count = user.public_metrics.followers_count

    # Get user's tweet count and average engagement rate
    tweets = api.get_timelines(user.id, max_results=100, tweet_fields=["attachments","author_id","context_annotations","created_at","entities","geo","in_reply_to_user_id","lang","public_metrics","reply_settings","source"])
    tweet_count = len(tweets.data)
    total_engagement = 0
    
    for tweet in tweets.data:
        total_engagement += tweet.public_metrics.like_count + tweet.public_metrics.retweet_count + tweet.public_metrics.quote_count + tweet.public_metrics.reply_count
        
    if tweet_count > 0:
        avg_engagement_rate = total_engagement / (tweet_count * follower_count) if total_engagement > 0 and tweet_count > 0 and follower_count > 0 else 0
    else:
        avg_engagement_rate = 0

    # Calculate influence score
    influence_score = math.log(follower_count + 1, 10) * (avg_engagement_rate + 1)
    
    return influence_score

### Reputation Calculus

In [80]:
def calculate_reputation(user: User):
    # Get user's recent mentions and replies
    mentions = api.search_tweets(query=f"@{user.username}", max_results=100)
    replies = api.search_tweets(query=f"to:{user.username}", max_results=100)

    # Calculate reputation score based on sentiment analysis of mentions and replies
    positive_sentiments = 0
    negative_sentiments = 0
    
    for mention in mentions.data:
        if mention.author_id != user.id:
            sentiment = get_sentiment_score(mention.text)
            if sentiment > 0:
                positive_sentiments += 1
            elif sentiment < 0:
                negative_sentiments += 1
                
    for reply in replies.data:
        if reply.author_id != user.id:
            sentiment = get_sentiment_score(reply.text)
            if sentiment > 0:
                positive_sentiments += 1
            elif sentiment < 0:
                negative_sentiments += 1
                
    if (positive_sentiments + negative_sentiments) > 0:
        reputation_score = positive_sentiments / (positive_sentiments + negative_sentiments)
    else:
        reputation_score = 0

    # Return influence and reputation scores
    return reputation_score

## Network Analysis

## Text Analysis

### Text Cleaning

In [294]:
import re
import string
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

def remove_urls(text):
    """Remove URLs from a text"""
    return re.sub(r"http\S+", "", text)

def remove_mentions(text):
    """Remove mentions from a text"""
    return re.sub(r"@\S+", "", text)

def remove_hashtags(text):
    """Remove hashtags from a text"""
    return re.sub(r"#\S+", "", text)

def remove_punctuation(text):
    """Remove punctuation from a text"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def tokenize(text):
    """Tokenize a text"""
    return word_tokenize(text, language='portuguese')

def remove_stopwords(tokens):
    """Remove stopwords from a list of tokens"""
    stop_words = set(stopwords.words('portuguese'))
    return [token for token in tokens if not token in stop_words]

def lemmatize(tokens):
    """Lemmatize a list of tokens"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for token in tokens:
        lemma = lemmatizer.lemmatize(token, wordnet.VERB)
        if lemma == token:
            lemma = lemmatizer.lemmatize(token, wordnet.NOUN)
        if lemma == token:
            lemma = lemmatizer.lemmatize(token, wordnet.ADJ)
        if lemma == token:
            lemma = lemmatizer.lemmatize(token, wordnet.ADV)
        lemmas.append(lemma)
    return lemmas

def synonymize(tokens):
    """Synonymize a list of tokens"""
    synonyms = []
    for token in tokens:
        synsets = wordnet.synsets(token, lang='por')
        if synsets:
            synset = synsets[0]
            for lemma in synset.lemmas(lang='por'):
                synonym = lemma.name()
                if synonym not in synonyms and synonym != token:
                    synonyms.append(synonym)
    return synonyms

def preprocess_tweet(text):
    """Preprocess a Brazilian Portuguese tweet"""
    text = text.lower()
    text = remove_urls(text)
    text = remove_mentions(text)
    text = remove_hashtags(text)
    text = remove_punctuation(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = lemmatize(tokens)
    tokens.extend(synonymize(tokens))
    return tokens

[nltk_data] Downloading package stopwords to C:\Users\Paulo
[nltk_data]     Roberto\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Paulo
[nltk_data]     Roberto\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Paulo
[nltk_data]     Roberto\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Paulo
[nltk_data]     Roberto\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Tweet Social Capital

In [314]:
import re
import datetime

def get_tweet_social_capital(tweet: Tweet):
    # Get tweet text
    text = tweet.text
    
    # Get number of likes and retweets
    likes = tweet.public_metrics.like_count
    retweets = tweet.public_metrics.retweet_count
    replies = tweet.public_metrics.reply_count
    quotes = tweet.public_metrics.quote_count
    hashtags = len(re.findall(r'#(\w+)', text))
    words = len(preprocess_tweet(text))
    
    words = words if hashtags > words else words * 5
    
    # Get tweet creation time
    created_at = tweet.created_at
    created_at = datetime.datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%S.%fZ")
    now = datetime.datetime.utcnow()
    age = (now - created_at).total_seconds() / 3600 # tweet age in hours
    
    # Calculate recency score
    recency_score = math.exp(-0.1 * age)
    
    # Get URLs in tweet
    urls = re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', text)
    num_urls = len(urls)
    
    # Get number of emojis in tweet
    emojis = re.findall(r'[^\w\s,]', text)
    num_emojis = len(emojis)
    
    # Get number of photos and videos in tweet
    num_medias = 0
    
    if tweet.attachments is not None and tweet.attachments.media_keys is not None:
        for attachment in tweet.attachments.media_keys:
            num_medias += 1
    
    # Calculate media score
    media_score = 0.5 * num_medias
    
    # Calculate URL score
    url_score = 0.5 * num_urls
    
    # Calculate emoji score
    emoji_score = 0.5 * num_emojis
    
    # Calculate engagement score
    engagement_score = likes + retweets + replies + quotes
    
    # Calculate social capital
    social_capital = recency_score * (media_score + url_score + emoji_score + engagement_score + hashtags + words)
    
    return {'tweet': tweet, 'score': social_capital }

# Social Capital Calculus

In [274]:
teste = {}
for user in public_tweets.includes.users:
    teste[user.id] = [calculate_influence(user), calculate_reputation(user)]
    

In [315]:
ranking = {}
usersTeste = {}
for tweet in public_tweets.data:
    for bla in public_tweets.includes.users:
        if(bla.id == tweet.author_id):
            usersTeste[bla.username] = sum(teste[tweet.author_id])
            
    ranking[tweet.id] = get_tweet_social_capital(tweet)
    ranking[tweet.id]['score'] += sum(teste[tweet.author_id]) 
    
    
rankedUsers = dict(sorted(usersTeste.items(), key=lambda item: item[1], reverse=True))
print(rankedUsers)

{'CNNBrasil': 7.152619498308262, 'radiobandnewsfm': 7.008128073673878, 'RevistaISTOE': 6.893159213031416, 'BomDiaBrasil': 6.716933460417127, 'jornaldarecord': 6.702706951781267, 'jornalodia': 6.623824474514334, 'g1rio': 6.455287867400853, 'correio24horas': 6.452392144546988, 'g1bahia': 5.752073780712917, 'falabrasil': 5.733561311945156, 'santos_agora': 5.7202895322360385, 'colaboraprojeto': 5.312955634844281, 'recordtvminas': 5.120710182075079, 'irmaospiologo': 5.097951440671946, 'g1alagoas': 3.977033684182141, 'g1zonadamata': 3.7368676117484414}


In [316]:
ranked = dict(sorted(ranking.items(), key=lambda item: item[1]['score'], reverse=True))
for tweet in ranked: 
    print(ranking[tweet]['score'], ranking[tweet]['tweet'].text)

246.58052393397892 #RESTAURAÇÃO 👷‍♂🏗 TEATRO MUNICIPAL ENTRA EM NOVA ETAPA DE OBRAS! A restauração das fachadas do conjunto arquitetônico do Centro de Cultura Patrícia Galvão se inicia agora, após o Carnaval. https://t.co/Xtdc5lHmSQ
245.17151266797143 'Não podemos controlar comportamento', diz Muquiranas após casos de assédio e vandalismo

Imagens de assédio e destruição envolvendo foliões do bloco foram divulgadas no Carnaval. Bloco diz que faz campanhas de conscientização https://t.co/Mx01FH96Zp #correio24h https://t.co/gfKYZAscu8
178.1962867821083 O Ministério dos Transportes ajudará o governo de São Paulo a reconstruir as rodovias que foram danificadas por conta das fortes chuvas durante o Carnaval. A determinação atende o pedido do governador do estado, Tarcísio de Freitas (Republicanos) #CNNNovoDia https://t.co/lojyEVYASu
168.17899019825933 Veja no #BomDiaBrasil: as buscas pelos desaparecidos em São Sebastião, no litoral de São Paulo, entram nesta quinta-feira (23) no 5º dia. E ma

# Results

## Influence vs Interactions

## Reputation vs Interactions

## Histogram Interactions

## Evaluation Metrics

In [49]:
def analyze_social_capital(tweets):
    public_tweets = api.search_tweets(query="stf lang:pt has:hashtags is:retweet", expansions=["referenced_tweets.id.author_id","in_reply_to_user_id","attachments.media_keys","author_id","entities.mentions.username"],
    user_fields=["created_at","entities","id","location","name","pinned_tweet_id","profile_image_url","protected","public_metrics","url","username","verified"],
    tweet_fields=["attachments","author_id","context_annotations","created_at","entities","geo","in_reply_to_user_id","lang","public_metrics","reply_settings","source"], max_results=100)

    api.get_followers()

    public_tweets.includes.tweets

#analyze_social_capital(public_tweets.data)


In [ ]:

def social_capital_impact(tweet, social_connections, interactions, texts, resources, user_popularity, user_influence):
  # extract the user, mentions, and hashtags from the tweet
  user = tweet['user']
  mentions = tweet['mentions']
  hashtags = tweet['hashtags']
  
  # compute the number of connections the user has in the social network
  user_connections = np.sum(social_connections[user])
  
  # compute the number of connections between the user and the mentions
  mention_connections = 0
  for mention in mentions:
    mention_connections += social_connections[user][mention]
  
  # compute the number of connections between the user and the hashtags
  hashtag_connections = 0
  for hashtag in hashtags:
    hashtag_connections += social_connections[user][hashtag]
  
  # compute the sum of the resources associated with the tweet
  resource_sum = 0
  for resource in resources:
    if resource in tweet['resources']:
      resource_sum += resources[resource]
  
  # compute the number of interactions for the tweet
  interaction_sum = interactions[tweet['tweet_id']]
  
  # compute the text similarity between the tweet and the other texts
  vectorizer = TfidfVectorizer()
  tweet_text = [tweet['text']]
  tweet_vector = vectorizer.fit_transform(tweet_text)
  other_vectors = vectorizer.transform(texts)
  similarity = cosine_similarity(tweet_vector, other_vectors)
  
  # compute the user popularity and influence factors
  popularity_factor = user_popularity[user]
  influence_factor = user_influence[user]
  
  # compute the social capital impact as the sum of mention connections, hashtag connections, resource sum, interaction sum,
  # text similarity, popularity factor, and influence factor divided by the number of user connections
  impact = (mention_connections + hashtag_connections + resource_sum + interaction_sum + similarity + popularity_factor + influence_factor) / user_connections
  
  return impact
